In [1]:
# training_data

#from rasa_nlu.training_data import load_data
from rasa_nlu.converters import load_data
#from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


trainer = Trainer(config.load("config_spacy.yml"))
training_data = load_data('training_data.json')
interpreter = trainer.train(training_data)



In [2]:
import spacy
from iexfinance.stocks import Stock
from iexfinance.stocks import get_historical_data
from iexfinance.stocks import get_historical_intraday
import sqlite3
import sys
import requests
import json
import matplotlib.pyplot as plt
import random
import re
import string

In [3]:
CONFUSE = -1
INIT = 0
MAIN = 1

CRT_PRICE = 2
HIS_PRICE = 3

In [4]:
response_group = {
   
    "greet":["What's up! What can I do for you?",
             "Hi! I'm ready to help you.",
            ],
    "finish":["Good bye!",
              "Hope to see you soon!",
             ],
    "function_intro":["Currently I am helping you with: \n1. Get stock information \n    1.1 Get current data \n    1.2 Get historical data \n    1.3 Analyze certain stocks \n2. Get weather information(every provience in China, seven days)"],

    
 
    "deny":["Sure, as you wish.\n\n{}",
            "OK, I'll figure it out.\n\n{}",
           ],
    
   
   
    "current_price":["The current price of {} is {}, and there are some news about {}:\n{}",
                     "{} has a real-time price of {}, and there are some news about {}:\n{}",
                    ], 
    "vague_historical_data":["Please specify which time of data you want to query.",
                             "Which time do you want to know?"
                            ],
    "analyze":["The Earning Per Share (TTM) of {} is currently {}."],
    
    
   
}
def resp_sentence(intent):
    return random.choice(response_group[intent])

In [5]:
policy_rules = {
  
    (INIT, "greet"): (MAIN, resp_sentence("greet"), None),
    (MAIN, "greet"): (MAIN, resp_sentence("greet"), None),
    (MAIN, "finish"): (MAIN, resp_sentence("finish"), None),
    
   
    (MAIN,"function_intro"): (MAIN, resp_sentence("function_intro"), None),

    
    (MAIN, "current_price"): (CRT_PRICE, resp_sentence("current_price"), None),

    (CRT_PRICE, "current_price"): (CRT_PRICE, resp_sentence("current_price"), None),

    (CRT_PRICE, "finish"): (MAIN, resp_sentence("finish"), None),
   

    (MAIN, "clear_historical_data"): (MAIN, "Here is a figure:", None),

    (MAIN, "vague_historical_data"): (MAIN, resp_sentence("vague_historical_data"), HIS_PRICE),
    (HIS_PRICE, "vague_historical_data"): (MAIN, resp_sentence("vague_historical_data"), HIS_PRICE),

    (MAIN, "add_historical_data"): (HIS_PRICE, "Here is a figure:", None),
    (HIS_PRICE, "vague_historical_data"): (HIS_PRICE, "Here is a figure:", None),

    (HIS_PRICE, "finish"): (MAIN, resp_sentence("finish"), None),

    (MAIN, "analyze"): (MAIN, resp_sentence("analyze"), None),
   
}

In [6]:

def send_message(state, pending, message):
   
    new_state, response, pending_state = respond(state, message)
   
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
    if pending_state is not None:
        pending = (pending_state, get_intent(message))
        
    return new_state, pending, response, get_intent(message)



weekday = []
city = ""

def respond(state, message):
    entity = get_entity(message)

    try:
        new_state = policy_rules[(state, get_intent(message))][0]

    except KeyError:
        new_state = CONFUSE
    pending_state = policy_rules[(state, get_intent(message))][2]
    
    

    if get_intent(message) == 'greet':
        response = policy_rules[(state, get_intent(message))][1]

    if get_intent(message) == 'finish':
        response = policy_rules[(state, get_intent(message))][1]
    

    if get_intent(message) == 'function_intro':
        response = policy_rules[(state, get_intent(message))][1]
        
    

    if get_intent(message) == 'current_price':
        response = policy_rules[(state, get_intent(message))][1].format(entity, get_current_price(entity), entity, get_news(entity))

    if get_intent(message) == 'clear_historical_data':
        response = policy_rules[(state, get_intent(message))][1]
        generate_figure(message)

    if get_intent(message) == 'vague_historical_data':
        response = policy_rules[(state, get_intent(message))][1]

    if get_intent(message) == 'add_historical_data':
        response = policy_rules[(state, get_intent(message))][1]
        generate_figure(message)

    if get_intent(message) == 'analyze':
        response = policy_rules[(state, get_intent(message))][1].format(entity,get_ttmEPS(entity))
 
def get_intent(message):
    return interpreter.parse(message)['intent']['name']

def get_entity(message):

    if interpreter.parse(message)['entities'] == []:
        return []

    if interpreter.parse(message)['entities'][0]['entity'] == 'company':
        return interpreter.parse(message)['entities'][0]['value']
  

        return [interpreter.parse(message)['entities'][0]['value'],
                interpreter.parse(message)['entities'][1]['value']]

In [7]:

def get_current_price(company):
    print("Company: ", company)
    
    prices = Stock(company).get_price()
    return prices


def get_ttmEPS(company):
    ttmEPS = Stock(company).get_key_stats()['ttmEPS']
    return ttmEPS

def get_news(company):
    news = Stock(company).get_news()
    for i in news:
        if i['summary'] != 'No summary available.':
            return i['url']

def generate_figure(message):
    comprehended_data = interpreter.parse(message)
  
    for i in range(0,2):

        if comprehended_data['entities'][i]['entity'] == 'company':
            required_company = comprehended_data['entities'][i]['value']
        

        if comprehended_data['entities'][i]['entity'] == 'his_price_type':
            required_type = comprehended_data['entities'][i]['value']
        

        else:
            required_company = 'AAPL'
            required_type = 'close'

    if len(comprehended_data['entities']) <= 3:

        time_period = [comprehended_data['entities'][0]['value'],
                       comprehended_data['entities'][1]['value']]

        start_time_splited = time_period[0].split(' - ')
        end_time_splited = time_period[1].split(' - ')

        start_year = int(start_time_splited[0])
        start_month = int(start_time_splited[1])
        start_day = int(start_time_splited[2])
        
        
        
  
        end_year = int(end_time_splited[0])
        end_month = int(end_time_splited[1])
        end_day = int(end_time_splited[2])

        start_time = datetime(start_year, start_month, start_day)
        end_time = datetime(end_year, end_month, end_day)
        

        his_data = get_historical_data(required_company,start_time,end_time,output_format='pandas')


    else:
        time_period = [comprehended_data['entities'][2]['value'],
                       comprehended_data['entities'][3]['value']]

        start_time_splited = time_period[0].split('-')
        end_time_splited = time_period[1].split('-')

          

        start_year = int(start_time_splited[0])
        start_month = int(start_time_splited[1])
        start_day = int(start_time_splited[2])
    

        end_year = int(end_time_splited[0])
        end_month = int(end_time_splited[1])
        end_day = int(end_time_splited[2])

        start_time = datetime(start_year, start_month, start_day)
        end_time = datetime(end_year, end_month, end_day)
        

        his_data = get_historical_data(required_company,start_time,end_time,output_format='pandas')
    

    plot_required_type = his_data[required_type].plot()
    fig = plot_required_type.get_figure()
    fig.savefig('result.png')

In [ ]:
while True:
    state = MAIN
    pending = None

    msg = input()
    print("USER: " + msg)
    state, pending, final_response, message_intent = send_message(state, pending, msg)

    print("BOT: " + final_response)
    